In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
# 数据预处理
transform = transforms.Compose([
    transforms.RandomRotation(degrees=20),            # 随机旋转图像，范围是 [-20度, 20度]
    transforms.RandomHorizontalFlip(p=0.5),           # 50%的概率水平翻转图像
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 随机平移，最大平移10%
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))              # 标准化
])

# 获取 MNIST 数据集
train_dataset = datasets.MNIST(root='/public/group_data_2023/luohh/Class/01.OmicsAndAI/01.Materials/homework/data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='/public/group_data_2023/luohh/Class/01.OmicsAndAI/01.Materials/homework/data', train=False, download=True, transform=transform)

# 数据加载器
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [3]:
class CNN(nn.Module):
    def __init__(self, num_conv_layers, kernel_size, num_channels):
        super(CNN, self).__init__()
        layers = []
        input_channels = 1
        
        # 创建卷积层
        for _ in range(num_conv_layers):
            layers.append(nn.Conv2d(input_channels, num_channels, kernel_size=kernel_size, padding=kernel_size//2))
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(2))  # 每个卷积后进行池化
            input_channels = num_channels
            
        self.features = nn.Sequential(*layers)
        
        # 计算卷积层之后尺寸的变化
        # 初始尺寸 28x28
        # 每个卷积层后面跟一个 2x2 的池化层，假设 kernel_size 和 padding 不会影响尺寸
        output_height = 28  # 每个卷积层后会减少 kernel_size 尺寸再进行池化
        for _ in range(num_conv_layers):
            output_height = (output_height - 2) // 2 + 1
        
        output_width = output_height  # MNIST 是正方形

        self.fc = nn.Linear(num_channels * output_height * output_width, 10)  # 确保这里的计算正确

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # 展平
        x = self.fc(x)
        return x

In [16]:
# 超参数范围
# num_conv_layers_options = [1, 2, 3]
num_conv_layers_options = [3]
# kernel_size_options = [3, 5, 7]
kernel_size_options = [5]
# num_channels_options = [4, 8, 16, 32]
num_channels_options = [32]
# learning_rate_options = [0.01, 0.0001, 0.00001]
learning_rate_options = [0.0001]
# epochs_options = range(1, 11)
epochs_options = [10]
batch_size_options = [16, 32, 64, 128]
# batch_size_options = [64]

results = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
# 遍历超参数组合
for num_conv_layers in num_conv_layers_options:
    for kernel_size in kernel_size_options:
        for num_channels in num_channels_options:
            for learning_rate in learning_rate_options:
                for epochs in epochs_options:
                    for batch_size in batch_size_options:
                        # 更新数据加载器的batch_size
                        print("### num_conv_layers = "+str(num_conv_layers)+", kernel_size = "+str(kernel_size)+", num_channels = "+str(num_channels)+
                              ", learning_rate = "+str(learning_rate)+", epochs = "+str(epochs)+", batch_size = "+str(batch_size))
                        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
                        test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

                        model = CNN(num_conv_layers, kernel_size, num_channels).to(device)
                        criterion = nn.CrossEntropyLoss()
                        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

                        # 训练模型
                        model.train()
                        for epoch in range(epochs):
                            for images, labels in train_loader:
                                images, labels = images.to(device), labels.to(device)
                                optimizer.zero_grad()
                                outputs = model(images)
                                loss = criterion(outputs, labels)
                                loss.backward()
                                optimizer.step()

                        # 评估模型
                        model.eval()
                        correct = 0
                        total = 0
                        with torch.no_grad():
                            for images, labels in test_loader:
                                images, labels = images.to(device), labels.to(device)
                                outputs = model(images)
                                _, predicted = torch.max(outputs.data, 1)
                                total += labels.size(0)
                                correct += (predicted == labels).sum().item()
                        
                        test_accuracy = correct / total
                        results.append((num_conv_layers, kernel_size, num_channels, learning_rate, epochs, batch_size, test_accuracy))
                        print("Accuracy = ",test_accuracy)
                        

### num_conv_layers = 3, kernel_size = 5, num_channels = 32, learning_rate = 0.0001, epochs = 10, batch_size = 16
Accuracy =  0.9595
### num_conv_layers = 3, kernel_size = 5, num_channels = 32, learning_rate = 0.0001, epochs = 10, batch_size = 32
Accuracy =  0.9522
### num_conv_layers = 3, kernel_size = 5, num_channels = 32, learning_rate = 0.0001, epochs = 10, batch_size = 64
Accuracy =  0.9328
### num_conv_layers = 3, kernel_size = 5, num_channels = 32, learning_rate = 0.0001, epochs = 10, batch_size = 128
Accuracy =  0.9145
